In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import groupby
import statistics
import copy
from langdetect import detect

In [3]:
df = pd.read_csv('../data/blogtext_selection.csv')
df.head()

,Unnamed: 0,id,gender,age,topic,sign,date,text
0,573,3176655,male,24,Engineering,Libra,"03,July,2004",If I had a band I'd call it '...
1,574,3176655,male,24,Engineering,Libra,"03,July,2004","Well... I hate to say it, bu..."
2,575,3176655,male,24,Engineering,Libra,"03,July,2004",How do we move on now? How d...
3,576,3176655,male,24,Engineering,Libra,"03,July,2004",Humor is my primary tool for ...
4,577,3176655,male,24,Engineering,Libra,"03,July,2004",Warning: If you know me and a...


In [4]:
authors = df['id'].values

print('Amount of blog posts:',len(df))
print('Number of authors:',len(set(authors)))

Amount of blog posts: 62697
Number of authors: 304


In [5]:
min_blogs = 100
min_blog_median = 1000

unique_authors = [a for a in set(authors)]
medians = []

for a in unique_authors:
    a_df = df[df['id'] == a]
    blogs = len(a_df)
    if blogs >= min_blogs:
        median_tuple = (statistics.median([len(t) for t in a_df['text'].values]),blogs,a)
        if median_tuple[0] >= min_blog_median:
            medians.append(median_tuple)
            
medians = sorted(medians, reverse=True)

candidate_auths = [p[2] for p in medians]
df_top = df[df['id'].map(lambda i: i in candidate_auths)]

print('Authors left after selection:',len(medians))
print('Number of blogs in selected dataset:',len(df_top))

Authors left after selection: 304
Number of blogs in selected dataset: 62697


In [5]:
selected_auths = []

for a in candidate_auths:
    a_df = df_top[df_top['id'] == a]
    blogs = a_df['text'].values
    
    english = True
    for b in blogs:
        if len(b) > 250:
            if detect(b) != 'en':
                english = False
    
    if english:
        selected_auths.append(a)

print('Remaining authors after language selection:', len(selected_auths))

Remaining authors after language selection: 304


In [6]:
df_selected = df_top[df_top['id'].map(lambda i: i in selected_auths)]
df_selected.to_csv('../data/blogtext_selection.csv')